# 07 - Visualization Library

**Purpose**: Create publication-quality charts for the presentation.

**Goal**: Generate 5-8 compelling visualizations that support your story.

**Tips**:
- Every chart should have a clear insight as the title
- Use colorblind-friendly palettes
- Large fonts (readable from back of room)
- Export as high-res PNG for slides

**Chart Ideas**:
1. Card win rate bar chart (Top 15)
2. Trophy distribution histogram (showing "walls")
3. Deck archetype performance comparison
4. Card synergy network diagram
5. Feature importance from model
6. Elixir cost vs win rate scatter
7. Trophy progression by arena

In [ ]:
import sys, os, pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, os.path.join(PROJECT_ROOT, 'src'))

from visualization import (
    setup_presentation_style, save_figure,
    plot_win_rate_comparison, plot_distribution,
    plot_scatter_with_regression, plot_correlation_heatmap
)

setup_presentation_style()

# Ensure output directory exists
os.makedirs(os.path.join(PROJECT_ROOT, 'presentation/figures'), exist_ok=True)

## Chart 1: [Your Key Insight]

Example: "Evolution Cards Dominate the Meta"

In [ ]:
# TODO: Load relevant data
# Example:
# card_stats = pd.read_parquet(os.path.join(PROJECT_ROOT, 'artifacts/card_win_rates.parquet'))

# Create chart
# fig = plot_win_rate_comparison(
#     card_stats,
#     'card_name',
#     'win_rate',
#     'Evolution Cards Dominate: Top 15 Cards by Win Rate',
#     top_n=15
# )

# Save for presentation
# save_figure('fig1_card_winrates.png')

print("TODO: Create Chart 1")

## Chart 2: [Your Second Insight]

In [ ]:
# TODO: Create second visualization
print("TODO: Create Chart 2")

## Chart 3: [Your Third Insight]

In [ ]:
# TODO: Create third visualization
print("TODO: Create Chart 3")

## Chart 4: Interactive Network (Optional)

Use Plotly for an interactive card synergy network (could be impressive in presentation!)

In [ ]:
# TODO: Create network visualization of card synergies
# Use networkx + plotly for interactive graph
print("TODO: Create network chart (optional but impactful!)")

## Export All Figures

Make sure all charts are saved to `presentation/figures/` as high-res PNGs

In [ ]:
# List all exported figures
import glob
figures = glob.glob(os.path.join(PROJECT_ROOT, 'presentation/figures/*.png'))
print(f"\n✓ Exported {len(figures)} figures:")
for fig in figures:
    print(f"  - {os.path.basename(fig)}")